# Clusterização

- Execute os algoritmos de agrupamento K-means e Hierárquico com diferentes valores de clusters e escolha o melhor resultado.

Para o hierárquico, varie entre o single e complete linkage e analise os resultados obtidos.

Para o k-means, inicialize os centroides aleatoriamente e avalie o impacto da inicialização nos resultados encontrados. 

Bônus: normalizar os dados e executar novamente os algoritmos para analisar os resultados gerados.
Fazer visualização dos clusters encontrados com PCA ou t-SNE.